<a id = 'top'></a>
# Design Patterns  
ref: https://blog.csdn.net/weicao1990/article/details/79108193  
ref: https://refactoringguru.cn/design-patterns  
* [Creational Patterns](#creational_patterns)
 * [Fattory Method](#factory_method)
 * [Abstract Factory](#abstract_factory)
 * [Builder](#builder)
 * [Prototype](#prototype)

<a id = 'creational_patterns'> </a>
## Creational Patterns 創建型模式 [top](#top)

<a id = 'factory_method'></a>
## Factory Method 工廠模式 [top](#top)
![title](design_pattern/factory_method.png)  
**基本概念：**  
* 在父類中提供一個創建對象的方法， 允許子類決定實例化對象的類型。

**適用性：**  
* 當一個父類不知道它所必須創建的對象的類的時候。  
* 當一個父類希望由它的子類來指定它所創建的對象的時候。  
* 當父類將創建對象的職責委托給多個幫助子類中的某一個，並且你希望將哪一個幫助子類是代理者這一信息局部化的時候。  


In [3]:
class ChinaGetter:
    """A simple localizer a la gettext"""
    def __init__(self):
        self.trans = dict(dog=u"小狗", cat=u"小猫")
 
    def get(self, msgid):
        """We'll punt if we don't have a translation"""
        try:
            return self.trans[msgid]
        except KeyError:
            return str(msgid)
        
class EnglishGetter:
    """Simply echoes the msg ids"""
    def get(self, msgid):
        return str(msgid)

def get_localizer(language="English"):
    """The factory method"""
    languages = dict(English=EnglishGetter, China=ChinaGetter)
    return languages[language]()
 
# Create our localizers
e, g = get_localizer("English"), get_localizer("China")
# Localize some text
for msgid in "dog parrot cat bear".split():
    print(e.get(msgid), g.get(msgid))

dog 小狗
parrot parrot
cat 小猫
bear bear


### 另一個實例問題

**問題:**  
假設你正在開發一款物流管理應用。最初版本只能處理卡車運輸， 因此大部分代碼都在位於名為卡車的類中。一段時間後，每天都能收到十幾次來自海運公司的請求，希望應用能夠支持海上物流功能。
![title](img/problem1-1.png)  
目前大部分代碼都與卡車類相關。在程式中添加輪船類需要修改全部代碼。如果你以後需要在程式中支持另外一種運輸方式， 很可能需要再次對這些代碼進行大幅修改。最後你將不得不編寫繁復的代碼，根據不同的運輸對象類，在應用中進行不同的處理。  

**解決方案:**  
工廠方法模式建議使用特殊的工廠方法代替對於對象構造函數的直接調用（即使用 new運算符）。 對象通過 new運算符創建，該運算符改在工廠方法中調用。工廠方法返回的對象通常被稱作 “產品”。  
![title](design_pattern/solution1-1.png)  

乍看之下， 這種更改可能毫無意義： 我們只是改變了程式中調用構造函數的位置而已。 但是現在你可以在子類中重寫工廠方法，從而改變其創建產品的類型。但有一點需要註意:僅當這些產品具有共同的基類或介面時，子類才能返回不同類型的產品，同時基類中的工廠方法還應將其返回類型聲明為這一共有介面。  
![title](design_pattern/solution1-2.png)  

舉例來說卡車Truck和 輪船Ship類都必須實現運輸Transport介面，該介面聲明一個名為 deliver交付的方法。每個類都將以不同的方式實現該方法： 
* 卡車走陸路交付貨物，輪船走海路交付貨物
* 陸路運輸Road Logistics類中的工廠方法返回卡車對象，海路運輸Sea Logistics類則返回輪船對象。  
![title](design_pattern/solution1-3.png)  

調用工廠方法的代碼（通常被稱為客戶端代碼）無需瞭解不同子類返回實際對象之間的差別。客戶端將所有產品視為抽象的運輸。客戶端知道所有運輸對象都提供交付方法，但是並不關心其具體實現方式。  

![title](design_pattern/factory_structure.png)  


In [5]:
from __future__ import annotations
from abc import ABC, abstractmethod

class Creator(ABC):
    """
    The Creator class declares the factory method that is supposed to return an object of a Product class. The Creator's subclasses usually provide the
    implementation of this method.
    """
    @abstractmethod
    def factory_method(self):
        """
        Note that the Creator may also provide some default implementation of the factory method.
        """
        pass

    def some_operation(self) -> str:
        """
        Also note that, despite its name, the Creator's primary responsibility is not creating products. Usually, it contains some core business logic
        that relies on Product objects, returned by the factory method. Subclasses can indirectly change that business logic by overriding the
        factory method and returning a different type of product from it.
        """

        # Call the factory method to create a Product object.
        product = self.factory_method()

        # Now, use the product.
        result = f"Creator: The same creator's code has just worked with {product.operation()}"

        return result

"""
Concrete Creators override the factory method in order to change the resulting product's type.
"""
class ConcreteCreator1(Creator):
    """
    Note that the signature of the method still uses the abstract product type,
    even though the concrete product is actually returned from the method. This
    way the Creator can stay independent of concrete product classes.
    """
    def factory_method(self) -> Product:
        return ConcreteProduct1()

class ConcreteCreator2(Creator):
    def factory_method(self) -> Product:
        return ConcreteProduct2()

class Product(ABC):
    """
    The Product interface declares the operations that all concrete products
    must implement.
    """
    @abstractmethod
    def operation(self) -> str:
        pass

"""
Concrete Products provide various implementations of the Product interface.
"""
class ConcreteProduct1(Product):
    def operation(self) -> str:
        return "{Result of the ConcreteProduct1}"

class ConcreteProduct2(Product):
    def operation(self) -> str:
        return "{Result of the ConcreteProduct2}"

def client_code(creator: Creator) -> None:
    """
    The client code works with an instance of a concrete creator, albeit through its base interface. As long as the client keeps working with the creator via
    the base interface, you can pass it any creator's subclass.
    """
    print(f"Client: I'm not aware of the creator's class, but it still works.\n"
          f"{creator.some_operation()}", end="")



print("App: Launched with the ConcreteCreator1.")
client_code(ConcreteCreator1())
print("\n")

print("App: Launched with the ConcreteCreator2.")
client_code(ConcreteCreator2())

App: Launched with the ConcreteCreator1.
Client: I'm not aware of the creator's class, but it still works.
Creator: The same creator's code has just worked with {Result of the ConcreteProduct1}

App: Launched with the ConcreteCreator2.
Client: I'm not aware of the creator's class, but it still works.
Creator: The same creator's code has just worked with {Result of the ConcreteProduct2}

<a id = 'abstract_factory'> </a>
## Abstract Factory 抽象工廠 [top](#top)

![title](design_pattern/abstract_factory.png)  
**基本概念**  
提供一個建立一系列相關或相互依賴對象的接口，而無需指定它們具體的類別。 

**適用性**
* 一個系統要獨立於它的產品的建立、組合和表示時。
* 一個系統要由多個產品系列中的一個來配置時。
* 當你要強調一系列相關的產品對象的設計以便進行聯合使用時。
* 當你提供一個產品類庫，而只想顯示它們的接口而不是實現時。

In [3]:
'''
Abstract Factory
'''
import random
 
class PetShop:
    """A pet shop"""
 
    def __init__(self, animal_factory=None):
        """pet_factory is our abstract factory.
        We can set it at will."""
        self.pet_factory = animal_factory
 
    def show_pet(self):
        """Creates and shows a pet using the
        abstract factory"""
        pet = self.pet_factory.get_pet()
        print("This is a lovely", str(pet))
        print("It says", pet.speak())
        print("It eats", self.pet_factory.get_food())

# Stuff that our factory makes
class Dog:
    def speak(self):
        return "woof"
 
    def __str__(self):
        return "Dog"

class Cat:
    def speak(self):
        return "meow"
 
    def __str__(self):
        return "Cat"

# Factory classes
class DogFactory:
    def get_pet(self):
        return Dog()
 
    def get_food(self):
        return "dog food"

class CatFactory:
    def get_pet(self):
        return Cat()
 
    def get_food(self):
        return "cat food"

# Create the proper family
def get_factory():
    """Let's be dynamic!"""
    return random.choice([DogFactory, CatFactory])()

# Show pets with various factories
if __name__ == "__main__":
    shop = PetShop()
    for i in range(3):
        shop.pet_factory = get_factory()
        shop.show_pet()
        print("=" * 20)

This is a lovely Dog
It says woof
It eats dog food
This is a lovely Dog
It says woof
It eats dog food
This is a lovely Dog
It says woof
It eats dog food


<a id = 'builder'></a>
## Builder建造者 [top](#top)

![title](design_pattern/builder.png)  
**意圖**  
將一個複雜對象的構建與它的表示分離，使得同樣的構建過程可以創建不同的表示。

**適用性**  
* 當創建覆雜對象的算法應該獨立於該對象的組成部分以及它們的裝配方式時。
* 當構造過程必須允許被構造的對象有不同的表示時。

In [4]:
"""
    Builder
"""
 
# Director
class Director(object):
    def __init__(self):
        self.builder = None
 
    def construct_building(self):
        self.builder.new_building()
        self.builder.build_floor()
        self.builder.build_size()
 
    def get_building(self):
        return self.builder.building

# Abstract Builder
class Builder(object):
    def __init__(self):
        self.building = None
 
    def new_building(self):
        self.building = Building()

# Concrete Builder
class BuilderHouse(Builder):
    def build_floor(self):
        self.building.floor = 'One'
 
    def build_size(self):
        self.building.size = 'Big'
 
 
class BuilderFlat(Builder):
    def build_floor(self):
        self.building.floor = 'More than One'
 
    def build_size(self):
        self.building.size = 'Small'

# Product
class Building(object):
    def __init__(self):
        self.floor = None
        self.size = None
 
    def __repr__(self):
        return 'Floor: %s | Size: %s' % (self.floor, self.size)

# Client
if __name__ == "__main__":
    director = Director()
    director.builder = BuilderHouse()
    director.construct_building()
    building = director.get_building()
    print(building)
    director.builder = BuilderFlat()
    director.construct_building()
    building = director.get_building()
    print(building)

Floor: One | Size: Big
Floor: More than One | Size: Small


<a id = 'prototype'></a>
## Prototype 原型 [top](#top)

![title](design_pattern/prototype.png)  
**意圖**  
用原型實例指定創建對象的種類，並且通過拷貝這些原型創建新的對象。

**適用性**  
當要實例化的類是在運行時刻指定時，例如，通過動態裝載；或者為了避免創建一個與產品類層次平行的工廠類層次時；或者當一個類的實例只能有幾個不同狀態組合中的一種時。建立相應數目的原型並克隆它們可能比每次用合適的狀態手工實例化該類更方便一些。

In [6]:
'''
Prototype
'''
 
import copy
 
class Prototype:
    def __init__(self):
        self._objects = {}

    def register_object(self, name, obj):
        """Register an object"""
        self._objects[name] = obj

    def unregister_object(self, name):
        """Unregister an object"""
        del self._objects[name]

    def clone(self, name, **attr):
        """Clone a registered object and update inner attributes dictionary"""
        obj = copy.deepcopy(self._objects.get(name))
        obj.__dict__.update(attr)
        return obj

def main():
    class A:
        def __str__(self):
            return "I am A"

    a = A()
    prototype = Prototype()
    prototype.register_object('a', a)
    b = prototype.clone('a', a=1, b=2, c=3)

    print(a)
    print(b.a, b.b, b.c)

if __name__ == '__main__':
    main()

I am A
1 2 3
